# Дипломчик
### Коды Гоппы

In [1]:
from sage.all import *
import random

In [33]:


class utils:
    def random_binary_unimodular_matrix(rows, cols):
        return random_matrix(GF(2), rows, cols, algorithm='unimodular')
    
    def random_permutation_matrix(n):
        return matrix(GF(2), Permutations(n).random_element().to_matrix())


    def random_irreducible_polynomial(R, degree):
        # var = R.random_element(degree=1).variables()[0] # get var of poly ring
        var = R.gen()

        c = 0
        while True: # todo: benchmark this! how fast we get irreducible poly
            res = 1*var^degree + R.random_element(degree=degree-1)
            c = c + 1

            if res.is_irreducible() == True:
                return res

        # print(f'iterations: {c}') 
        
        return None

    def random_binary_vector(length):
        return vector(random_matrix(GF(2), 1, length))

    def random_binary_err_vector(length, weight_bound):
        list = [0]*length

        weight = 0
        for i in range(length):
            if weight == weight_bound:
                break
        
            # rand_elem = GF(2).random_element() # works bad, gets only null-vec or 't' 1s in the head of vec
            
            rand_elem = random.randint(0,1)
            
            list[i] = rand_elem
            weight = weight + rand_elem

        return vector(GF(2), list)

    def solve_linear_eq(A, b):
        #aug = A.augment(b, subdivide=True)
        #return aug.rref()
        return A.solve_left(b)

    def get_syndrome(R, L, g, y):
        if len(L) != len(y):
            raise Exception("len(L) != len(y)")

        var = R.gen()
        syndrome = 0
        for i in range(len(y)):
            syndrome += y[i] / (var - L[i])

        # todo: return syndrome mod g(x)

        return syndrome.quo_rem(g)[0] # todo: is correct??

        

In [56]:
F = GF(2^4, 'a')

R.<x> = F[]

p1 = x^4 - 3*x^3 + 2*x 
p2 = x^3 + 7

print(p1.quo_rem(p2))

# print(R.random_element(degree=1).variables()[0]) # get var of poly ring
# utils.random_irreducible_polynomial(R, 4)

S = R.quotient(p2, 'x')
print(S)

print(p1.mod(p2))

print(S(x)^(-1))

(d,u,v) = xgcd(p2,p1);
print(u.mod(p2))


(x + 1, x + 1)
Univariate Quotient Polynomial Ring in x over Finite Field in a of size 2^4 with modulus x^3 + 1
x + 1
x^2
x + 1


In [34]:
F = GF(2^4, 'a')

print(f"elems of field F:\n")
for i,x in enumerate(F):  print("{} {}".format(f'{i+1}:', x))

R.<x> = F[]
RR.<a> = F

t = 2 # todo: understand how t related to p^m

g = utils.random_irreducible_polynomial(R, t)
# g = x^2 + x + a^3

L = [a for a in F.list() if g(a) != 0]
#print(f"elems of L:\n")
#for i,l in enumerate(L):  print("{} {}".format(f'{i+1}:', l))

C = codes.GoppaCode(g, L)

# n, k, t - publicly available parameter
n = len(L)
k = C.dimension()

print(f'\nn: {n}\nk: {k}\nt: {t}\n')

print(f'{C}\n')

# n - размерность векторного пространства над которым строится код, n зависит только от L
# k - размерность пространства кода, для кодов Гоппы справедливо: k >= n - mt
# d - минимальное расстояние кода, для кодов Гоппы справедливо: d >= t + 1, для неприводимого бинарного кода Гоппы: d >= 2t + 1

H = C.parity_check_matrix()
G = C.generator_matrix()

print(f'check matrix:\n{H}\n')
print(f'gen matrix: \n{G}\n')

r = len(G.rows())
c = len(G.columns())

S = utils.random_binary_unimodular_matrix(r, r)
print(f'random binary matrix:\n{S}\n')

P = utils.random_permutation_matrix(c)
print(f'random permutation matrix:\n{P}\n')

GG = S * G * P
print(f"G':\n{GG}\n")

# public key: (G', t)
# private key: (g(x), G, S, P)

# bob sends to alice plaintext m:

m = utils.random_binary_vector(k)
print(f'message: {m}\n')

e = utils.random_binary_err_vector(n, 0) # todo: wt(e) <= t!!
print(f'err vec: {e}\n')

c = (m * GG) + e
print(f'cipher text: {c}\n')

# alice decrypts received ciphertext c:

# c * P^(-1) = m * S * G + e * P^(-1) = m * S * G + e'
# then use Patterson algo to find e' = e * P^(-1)
# then find c' - e' = m * S * G
# L: m' = mS => solve m'G = c' - e' and thus find m'
# then find m = m'S^(-1)

mSS = G.solve_left(c * P.inverse())
print(f"mS':\n{mSS}\n")

decrypted = mSS * S.inverse()
print(f"decrypted message:\n{decrypted}\n")

print(f"are equal: {decrypted == m}")

print(f"syndrome: {utils.get_syndrome(R, L, g, c)}")

elems of field F:

1: 0
2: a
3: a^2
4: a^3
5: a + 1
6: a^2 + a
7: a^3 + a^2
8: a^3 + a + 1
9: a^2 + 1
10: a^3 + a
11: a^2 + a + 1
12: a^3 + a^2 + a
13: a^3 + a^2 + a + 1
14: a^3 + a^2 + 1
15: a^3 + 1
16: 1

n: 16
k: 8
t: 2

[16, 8] Goppa code over GF(2)

check matrix:
[1 0 1 0 1 1 1 1 0 0 0 1 0 1 1 1]
[0 1 1 0 1 0 1 0 0 1 1 1 1 0 1 1]
[1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 1]
[1 0 0 1 1 0 1 0 1 0 0 1 0 1 0 1]
[0 0 1 0 0 1 0 1 1 1 1 0 1 0 0 1]
[0 0 1 1 1 0 0 0 0 0 0 0 0 1 1 1]
[0 1 1 0 1 1 0 0 0 0 0 0 1 1 0 1]
[0 0 1 1 1 1 1 0 1 1 0 1 1 1 1 1]

gen matrix: 
[1 0 0 0 0 0 0 1 0 1 1 0 1 1 1 0]
[0 1 0 0 0 0 0 1 0 1 0 1 0 1 1 0]
[0 0 1 0 0 0 0 1 0 1 0 1 0 0 0 1]
[0 0 0 1 0 0 0 0 0 1 1 1 0 0 1 0]
[0 0 0 0 1 0 0 1 0 0 0 1 1 0 1 0]
[0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 1]
[0 0 0 0 0 0 1 0 0 0 1 1 0 1 0 1]
[0 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1]

random binary matrix:
[1 0 1 0 0 0 1 1]
[1 1 1 0 1 0 1 1]
[1 1 0 1 0 0 1 0]
[1 1 1 1 1 1 0 0]
[0 1 0 0 0 1 0 1]
[1 1 1 1 0 1 1 0]
[0 0 0 1 1 1 1 1]
[0 1 0 1 0 0 0 1]

rando